# Coleta e Pré-processamento dos Dados

Neste notebook, utilizaremos a biblioteca `yfinance` para coletar dados históricos de preços de ações e realizaremos o pré-processamento necessário para preparar os dados para o modelo LSTM.

In [ ]:
import yfinance as yf
import pandas as pd

# Especifique o símbolo da empresa que você vai trabalhar
symbol = 'DIS'  # Exemplo: Disney

# Configure data de início e fim da sua base
start_date = '2018-01-01'
end_date = '2024-07-20'

# Use a função download para obter os dados
df = yf.download(symbol, start=start_date, end=end_date)

# Exibir as primeiras linhas do dataframe
df.head()

## Pré-processamento dos Dados

Aqui, realizaremos o pré-processamento dos dados, incluindo a normalização e a criação de conjuntos de dados para treinamento e teste.

In [ ]:
# Normalização dos dados
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

# Criar conjuntos de dados para treinamento e teste
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Função para criar conjuntos de dados com base em um tamanho de janela
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Definir o tamanho da janela
time_step = 60

# Criar os conjuntos de dados de treinamento e teste
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape os dados para o formato [amostras, tempo, características]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Salvar os dados pré-processados em um arquivo CSV
pd.DataFrame(scaled_data, columns=['Close']).to_csv('../data/dataset.csv', index=False)